In [1]:
import pandas as pd
import pyarrow.parquet as pq

ModuleNotFoundError: No module named 'pyarrow'

: 

In [4]:
parquetDf = pq.ParquetDataset('data/')
eventsDf = parquetDf.read().to_pandas()

In [5]:
aidsGrouped = eventsDf.groupby(['aid', 'type'])['aid', 'type'].count()
aidsClicks = aidsGrouped[aidsGrouped['type'] == 'clicks']
aidsCarts = aidsGrouped[aidsGrouped['type'] == 'carts']
aidsOrders = aidsGrouped[aidsGrouped['type'] == 'orders']

/var/folders/h0/ghgtz2c94bj8xyjcn3n5xvh80000gn/T/ipykernel_3756/187901066.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  aidsGrouped = eventsDf.groupby(['aid', 'type'])['aid', 'type'].count()


In [6]:
#EVENT DATA
nevents = len(eventsDf)
types = pd.unique(eventsDf['type'])
nAids = len(aidsGrouped)
print("Events:", nevents, "Types:", types, 'NProducts:', nAids)
clicksEv = eventsDf[eventsDf['type'] == 'clicks'].shape[0]
cartsEv = eventsDf[eventsDf['type'] == 'carts'].shape[0]
ordersEv = eventsDf[eventsDf['type'] == 'orders'].shape[0]

print("ClicksEvents:", clicksEv, str((clicksEv/nevents)*100).format(':%'))
print("CartsEvents:", cartsEv, str((cartsEv/nevents)*100).format(':%'))
print("OrdersEvents:", ordersEv, str((ordersEv/nevents)*100).format(':%'))




Events: 216716096 Types: ['clicks' 'carts' 'orders'] NProducts: 3748278
ClicksEvents: 194720954 89.85071141185563
CartsEvents: 16896191 7.796463350834817
OrdersEvents: 5098951 2.3528252373095535
